In [1]:
import os
import urllib.parse
import logging
import jupyter_black
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [5]:
jupyter_black.load()
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

In [6]:
def get_random_page_urls(driver: webdriver.Chrome, languages: list) -> dict:
    """
    Récupère les URLs de pages Wikipédia aléatoires pour une liste de langues données.

    Parameters:
        driver (webdriver.Chrome): Instance du navigateur Chrome.
        languages (list): Liste de codes de langues Wikipédia.

    Returns:
        dict: Dictionnaire contenant les URLs des pages aléatoires pour chaque langue.
    """
    urls = {}
    for language in languages:
        driver.get(f"https://{language}.wikipedia.org/wiki/Special:Random")
        urls[language] = driver.current_url
    return urls

Je commence par définir une fonction nommée `get_random_page_urls` qui prend deux paramètres : `driver` et `languages`. Le `driver` est probablement un objet qui me permet d'interagir avec un navigateur web, et `languages` est une liste des codes de langue pour lesquels je veux récupérer des URL aléatoires de pages Wikipédia.

À l'intérieur de la fonction, je crée un dictionnaire vide appelé `urls` où je vais stocker les URL que je récupère. Pour chaque langue présente dans la liste `languages`, je fais plusieurs actions :

1. Je construis l'URL de la page Wikipédia aléatoire pour cette langue en utilisant la méthode `f`. Cela me donne une URL comme 'https://en.wikipedia.org/wiki/Special:Random' si la langue est 'en' (anglais), par exemple.

2. J'utilise le `driver` pour charger cette URL dans le navigateur web avec `driver.get(url_formée)`.

3. Une fois que la page a chargé, je récupère l'URL actuelle, qui est une page spécifique choisie aléatoirement par Wikipédia, en utilisant `driver.current_url`.

4. Je stocke cette URL dans le dictionnaire `urls` avec la langue correspondante comme clé.

Après avoir traité toutes les langues dans la boucle, je retourne le dictionnaire `urls` qui contient les codes de langue comme clés et les URL des pages Wikipédia aléatoires comme valeurs.

In [7]:
def get_interlanguage_urls(driver, base_url: str) -> dict:
    """
    Obtient les URLs des pages correspondantes dans d'autres langues pour une page donnée

    Parameters:
    driver (WebDriver): Le driver Selenium
    base_url (str): L'URL de la page pour laquelle on veut récupérer les URLs des pages correspondantes dans d'autres langues

    Returns:
    dict: Un dictionnaire avec les langues en clés et les URLs en valeurs
    """
    driver.get(base_url)
    try:
        interlanguage_links = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "li.interlanguage-link a")
            )
        )

        return {
            link.get_attribute("lang"): link.get_attribute("href")
            for link in interlanguage_links
        }

    except TimeoutException:
        logging.error(
            f"Timeout when trying to find interlanguage links for {base_url}\n"
        )
        return {}

Je commence par importer le module `logging` qui me permet d'enregistrer des informations sur ce qui se passe dans mon code. Je configure le niveau de logging à `INFO`, ce qui signifie que je vais enregistrer les messages de niveau INFO et supérieur (comme ERROR). Je définis également un format spécifique pour ces messages qui inclut la date et l'heure, le niveau de gravité du message et le message lui-même.

Ensuite, je définis une fonction nommée `get_interlanguage_urls` qui prend deux paramètres : `driver` et `base_url`. Le `driver` est l'objet qui interagit avec le navigateur, et `base_url` est l'URL de la page de base à partir de laquelle je veux récupérer des liens interlangues.

À l'intérieur de la fonction, je fais plusieurs actions :

1. Je charge l'URL de base dans le navigateur en utilisant `driver.get(base_url)`.

2. Je crée un objet `wait` de type `WebDriverWait`, lié au `driver` et avec un délai maximal de 30 secondes. Cet objet me permet d'attendre que certaines conditions soient remplies avant de continuer l'exécution de mon code.

3. J'essaie ensuite de récupérer tous les éléments de la page qui correspondent à des liens interlangues. Ces liens sont localisés par un sélecteur CSS qui cible les éléments 'a' dans les 'li' avec la classe 'interlanguage-link'. J'attends que ces éléments soient présents sur la page, ce que je vérifie avec la méthode `presence_of_all_elements_located`.

4. Si les éléments sont trouvés à temps, je construis un dictionnaire avec les langues comme clés (en récupérant l'attribut 'lang' de chaque lien) et les URL correspondantes comme valeurs (en récupérant l'attribut 'href' de chaque lien).

5. Si le délai d'attente de 30 secondes est dépassé sans que les liens soient trouvés, je consigne une erreur de type `TimeoutException` avec un message d'erreur spécifique qui mentionne l'URL de base. Dans ce cas, je retourne un dictionnaire vide.

La fonction `get_interlanguage_urls` retourne donc soit un dictionnaire contenant les langues et leurs URL associées si tout se passe bien, soit un dictionnaire vide si une erreur se produit lors de la récupération des liens.

In [8]:
def extract_paragraphs(driver: webdriver.Chrome, url: str) -> tuple:
    """
    Extrait les paragraphes d'un article Wikipédia

    Parameters:
    driver (WebDriver): Le driver Selenium
    url (str): L'URL de l'article Wikipédia

    Returns:
    str: Les paragraphes de l'article
    str: Le titre de l'article
    """
    driver.get(url)

    paragraphs = ""

    try:
        title = os.path.basename(urllib.parse.urlparse(url).path)
        elements = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div/p')

        print(
            f"Found {len(elements)} paragraphs for {url} : {[el.text[:30] for el in elements]}"
        )

        for el in elements:
            if el.text:
                paragraphs += el.text + "\n"
            else:
                continue

    except:
        print(f"Could not find paragraph for {url}")

    return paragraphs, title

Je définis ici une fonction appelée `extract_paragraphs` qui prend deux paramètres : `driver` et `url`.

Voici ce que je fais dans cette fonction :

1. Je commence par charger l'URL donnée dans le navigateur à l'aide de `driver.get(url)`.

2. Je déclare une variable `paragraphs` que j'initialise avec une chaîne de caractères vide. Cela servira à stocker tous les paragraphes de texte que je vais extraire.

3. Je tente ensuite d'extraire le premier paragraphe de l'article. Pour cela, je procède comme suit :

   a. J'extrais le titre de la page à partir de l'URL en utilisant les fonctions `os.path.basename` et `urllib.parse.urlparse` pour obtenir uniquement la partie finale de l'URL, qui correspond généralement au titre de l'article.

   b. Je recherche dans le document tous les éléments qui correspondent au chemin XPath spécifié. Ce chemin cible les paragraphes situés dans une division particulière de la page identifiée par l'identifiant `mw-content-text`.

   c. J'affiche un message indiquant combien de paragraphes j'ai trouvés pour l'URL donnée. Pour chaque élément trouvé, je montre les 30 premiers caractères du texte de cet élément.

   d. Je parcours ensuite chaque élément trouvé. Si l'élément contient du texte, je l'ajoute à la variable `paragraphs` avec un saut de ligne à la fin. Si l'élément ne contient pas de texte, je continue simplement sans l'ajouter.

4. Si une exception se produit pendant l'extraction (par exemple, si le chemin XPath est incorrect ou si la page ne se charge pas correctement), j'affiche un message d'erreur indiquant que je n'ai pas pu trouver de paragraphe pour l'URL donnée.

5. Enfin, je retourne un tuple contenant les paragraphes extraits et le titre de l'article. Cela permet à celui qui utilise cette fonction de recevoir à la fois le texte extrait et le titre de la page à partir de laquelle le texte a été extrait.

In [9]:
def write_txt(
    data_folder: str, language: str, url: str, paragraph: str, file_counts: dict
) -> None:
    """
    Ecrit un paragraphe dans un fichier texte

    Parameters:
    data_folder (str): Dossier de destination des fichiers.
    language (str): Code de la langue de l'article.
    url (str): URL de l'article.
    paragraph (str): Texte de l'article.
    file_counts (dict): Dictionnaire contenant le nombre de fichiers écrits pour chaque langue.

    Returns:
    None
    """
    parsed_url = urllib.parse.urlparse(url)
    article_title = os.path.basename(parsed_url.path)
    article_title = urllib.parse.unquote(article_title)
    sanitized_title = "".join(char if char.isalnum() else "_" for char in article_title)

    os.makedirs(data_folder, exist_ok=True)
    filename = f"{language}_{sanitized_title}.txt"
    file_path = os.path.join(data_folder, filename)

    try:
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(paragraph)
        file_counts[language] += 1
        logging.info(f"Successfully wrote {file_path}")
    except OSError as e:
        logging.error(f"Failed to write {file_path}: {e}")

Je définis une fonction nommée `write_txt` qui prend cinq paramètres : `data_folder`, `lang`, `url`, `paragraph`, et `file_counts`. Ces paramètres me servent respectivement à spécifier où sauvegarder les fichiers, la langue de l'article, l'URL de l'article, le paragraphe à écrire, et un dictionnaire pour suivre combien de fichiers j'ai écrits pour chaque langue.

Voici comment je procède dans cette fonction :

1. **Extraction du titre de l'article depuis l'URL** : 
   - Je parse l'URL pour isoler la partie qui me sert de titre, qui est après le dernier '/' dans le chemin de l'URL.
   - Je décode le titre pour convertir des séquences d'échappement URL (comme '%E1%83%90') en caractères lisibles.

2. **Nettoyage du titre** :
   - Je remplace tout caractère du titre qui n'est pas alphanumérique par un underscore ('_') pour m'assurer que le titre peut être utilisé comme nom de fichier sans causer d'erreur.

3. **Préparation du dossier de données** :
   - Je m'assure que le dossier où je veux sauvegarder les fichiers existe. S'il n'existe pas, je le crée avec `os.makedirs`, et l'option `exist_ok=True` me permet de ne pas générer d'erreur si le dossier existe déjà.

4. **Écriture du paragraphe dans un fichier** :
   - Je génère le nom du fichier en utilisant la langue et le titre de l'article nettoyé, et j'ajoute l'extension '.txt'.
   - Je combine le dossier de données et le nom du fichier pour créer le chemin complet du fichier.
   - J'ouvre le fichier en mode écriture (`'w'`) avec encodage UTF-8, et j'écris le paragraphe dedans. Je termine par afficher un message pour confirmer que l'écriture a réussi.

5. **Incrémentation du compteur de fichiers** :
   - J'augmente de 1 le compteur pour la langue correspondante dans le dictionnaire `file_counts` pour suivre combien de fichiers ont été écrits pour chaque langue.

La fonction `write_txt` n'a pas de valeur de retour. Elle effectue son travail d'écriture en sauvegardant les paragraphes dans des fichiers spécifiques tout en organisant et en comptant combien de fois elle le fait pour chaque langue.

In [10]:
def main():
    driver = webdriver.Chrome()
    data_folder = "../../../data/raw/"
    languages = ["ar", "fr", "en", "de", "es", "ja", "ko", "ru", "zh"]
    file_counts = {lang: 0 for lang in languages}

    base_urls = get_random_page_urls(driver, languages)
    for lang in languages:
        while file_counts[lang] < 100:
            current_url = base_urls[lang]
            paragraphs, article_title = extract_paragraphs(driver, current_url)
            if paragraphs:
                write_txt(data_folder, lang, article_title, paragraphs, file_counts)

            links = get_interlanguage_urls(driver, current_url)
            for link_lang, link_url in links.items():
                if link_lang in languages and file_counts[link_lang] < 100:
                    link_paragraphs, title = extract_paragraphs(driver, link_url)
                    if link_paragraphs:
                        write_txt(
                            data_folder, link_lang, title, link_paragraphs, file_counts
                        )

            new_urls = get_random_page_urls(driver, [lang])
            base_urls[lang] = new_urls[lang]

    driver.quit()

J'ai défini une fonction nommée `main` qui orchestre l'extraction et l'enregistrement de textes à partir de différentes éditions linguistiques de Wikipedia. Voici comment je m'y prends :

1. **Initialisation du pilote du navigateur** :
   - J'initialise un pilote Chrome en utilisant `webdriver.Chrome()`. Cela me permet d'ouvrir et de manipuler un navigateur Chrome directement depuis mon script.

2. **Définition du dossier de données et des langues** :
   - Je définis le chemin vers le dossier où je souhaite enregistrer les fichiers extraits. Je spécifie également une liste de codes de langue pour les éditions de Wikipedia que je souhaite explorer.

3. **Initialisation des compteurs de fichiers** :
   - Je crée un dictionnaire `file_counts` pour suivre le nombre de fichiers que j'ai écrits pour chaque langue, en initialisant tous les compteurs à zéro.

4. **Récupération des URL initiales** :
   - J'utilise la fonction `get_random_page_urls` pour obtenir une URL de page aléatoire pour chaque langue listée. Cela me donne un point de départ pour chaque langue.

5. **Traitement pour chaque langue** :
   - Pour chaque langue, je continue à extraire et enregistrer des paragraphes tant que je n'ai pas atteint 20 fichiers pour cette langue. Je récupère l'URL actuelle pour la langue en question et j'extrait des paragraphes et le titre de l'article à partir de cette URL.

6. **Écriture des paragraphes** :
   - Si des paragraphes sont extraits, je les enregistre en utilisant la fonction `write_txt`.

7. **Exploration des liens interlangues** :
   - J'utilise `get_interlanguage_urls` pour récupérer des liens vers d'autres éditions linguistiques de la même page Wikipedia. Pour chaque lien valide qui pointe vers une langue que je suis en train de traiter et où je n'ai pas encore atteint la limite de 20 fichiers, j'extrait et j'enregistre les paragraphes.

8. **Rafraîchissement des URL** :
   - Après avoir traité les liens interlangues, je rafraîchis l'URL de base pour la langue actuelle en récupérant une nouvelle URL aléatoire. Cela me permet de continuer à extraire de nouveaux contenus sans boucler sur les mêmes pages.

9. **Nettoyage** :
   - Une fois que j'ai fini de traiter toutes les langues, je ferme le pilote du navigateur avec `driver.quit()` pour libérer les ressources.

Cette fonction `main` organise donc systématiquement l'extraction de contenus multilingues à partir de Wikipedia, les sauvegarde localement, et assure que le nombre de fichiers pour chaque langue atteint un seuil spécifié. Cela me permet de construire une collection diversifiée de textes pour différents usages, comme l'entraînement de modèles de traitement de langues.

In [11]:
main()

Found 1 paragraphs for https://ar.wikipedia.org/wiki/%D8%A3%D9%84%D9%81%D8%B1%D8%AF_%D9%88%D9%88%D9%83%D8%B1_(%D9%85%D8%A8%D8%A7%D8%B1%D8%B2_%D8%A8%D8%A7%D9%84%D8%B3%D9%8A%D9%81) : ['ألفرد ووكر (بالإنجليزية: Alfre']
Found 2 paragraphs for https://ar.wikipedia.org/wiki/%D9%84%D9%8A%D8%AA%D8%B4%D8%A7%D8%AA%D9%8A%D9%84%D9%8A%D8%B1%D9%8A%D8%AA : ['الليتشاتيليريت هو شكل زجاجي من', 'يصنف الليتشاتيليريت على أنه شب']
Found 8 paragraphs for https://ar.wikipedia.org/wiki/%D8%B3%D9%82%D9%88%D8%B7_%D8%A7%D9%84%D9%85%D8%B1%D8%B3%D9%89_%D8%A7%D9%84%D9%83%D8%A8%D9%8A%D8%B1_(1505) : ['الاستيلاء على مرسى الكبير بالأ', 'بعد غزو غرناطة في أسبانيا وقمع', 'تم تجهير أسطول مكون من مائة وس', 'غادرت القوات البحرية ميناء مال', 'وفي الجزائر علموا بالحملة الأس', 'تم الهجوم في 11 سبتمبر عندما أ', 'في اليوم التالي في 12 سبتمبر ه', 'بقي المرسى حوالي 300 عام تحت ا']
Found 6 paragraphs for https://ar.wikipedia.org/wiki/%D9%87%D9%8A%D9%84%D8%AF%D9%8A%D8%BA%D8%A7%D8%B1%D8%AF_%D9%84%D9%8A%D8%B4%D8%B1%D8%AA : ['هيلدغارد م

ERROR:root:Timeout when trying to find interlanguage links for https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%B9%D9%84%D8%A7%D9%82%D8%A7%D8%AA_%D8%A7%D9%84%D8%AA%D9%88%D8%BA%D9%88%D9%8A%D8%A9_%D8%A7%D9%84%D9%81%D9%8A%D8%AA%D9%86%D8%A7%D9%85%D9%8A%D8%A9



Found 4 paragraphs for https://ar.wikipedia.org/wiki/%D8%A3%D9%84%D9%88%D9%8A%D8%B3_%D8%A8%D8%B1%D9%88%D9%86%D8%B1 : ['أليوس بورنير (بالألمانية: Aloi', 'عاش أليوس بورنير في فترة الحكم', 'كان برونير خرج من ألمانيا خوفا', 'توفي في العاصمة السورية دمشق ل']
Found 4 paragraphs for https://ar.wikipedia.org/wiki/%D9%83%D8%A7%D8%A6%D9%86_%D8%A3%D8%B3%D8%B7%D9%88%D8%B1%D9%8A : ['المخلوق الأسطوري أو الكائن الخ', 'ومن ناحية أخرى كانت العديد من ', 'تظهر مجموعة من الحيوانات في ال', 'بعض مخلوقات العصر الكلاسيكي، م']
Found 1 paragraphs for https://ar.wikipedia.org/wiki/%D9%83%D9%8A%D9%8A%D9%84_%D8%B3%D9%81%D9%8A%D9%86%D8%B3%D9%88%D9%86 : ['كييل سفينسون (بالسويدية: Kjell']
Found 1 paragraphs for https://ar.wikipedia.org/wiki/%D8%AC%D8%B1%D8%A7%D8%A8_%D8%A7%D9%84%D8%A3%D8%B9%D9%84%D9%89_(%D8%A7%D9%84%D9%85%D8%B3%D9%8A%D9%85%D9%8A%D8%B1) : ['جراب الأعلى هي إحدى قرى عزلة ا']


ERROR:root:Timeout when trying to find interlanguage links for https://ar.wikipedia.org/wiki/%D8%AC%D8%B1%D8%A7%D8%A8_%D8%A7%D9%84%D8%A3%D8%B9%D9%84%D9%89_(%D8%A7%D9%84%D9%85%D8%B3%D9%8A%D9%85%D9%8A%D8%B1)



Found 1 paragraphs for https://ar.wikipedia.org/wiki/%D9%83%D8%A7%D8%B1%D9%88%D9%84_%D8%B3%D9%88%D8%AA%D9%88%D9%86_(%D8%B1%D8%B3%D8%A7%D9%85%D8%A9) : ['كارول سوتون (بالإنجليزية: Caro']
Found 2 paragraphs for https://ar.wikipedia.org/wiki/%D8%AF%D9%8A_%D8%A3%D9%83%D8%B3%D8%AC%D9%8A%D9%86 : ['دي أكسجين (بالإنجليزية: Doxyge', 'Doxygen هو برنامج مجاني، تم إص']
Found 1 paragraphs for https://ar.wikipedia.org/wiki/%D8%B2%D8%A7%D9%86%D8%AB%D9%88%D8%B3%D9%8A%D9%86_%D8%AB%D9%84%D8%A7%D8%AB%D9%8A_%D8%A7%D9%84%D9%81%D8%B3%D9%81%D8%A7%D8%AA : ['الزانثوسين ثلاثي الفسفات (بالإ']
Found 1 paragraphs for https://ar.wikipedia.org/wiki/%D9%81%D9%8A%D9%84%D9%8A%D8%A8_%D8%A8%D9%88%D9%86%D8%AF : ['فيليب بوند (بالإنجليزية: Phili']
Found 1 paragraphs for https://ar.wikipedia.org/wiki/%D8%A3%D9%88%D8%B3%D9%83%D8%A7%D8%B1_%D9%81%D9%88%D8%B3%D8%AA%D8%B1 : ['أوسكار فوستر (2 ديسمبر 1874 في']
Found 1 paragraphs for https://ar.wikipedia.org/wiki/%D8%A8%D8%A7%D8%AA%D8%B1%D9%8A%D9%83_%D8%A3%D9%83%D9%86%D9%8A%D9%84 : [

...

Found 2 paragraphs for https://ar.wikipedia.org/wiki/%D9%83%D9%8A%D8%AB_%D8%AF%D9%8A%D9%81%D9%8A%D8%B3 : ['كيث ديفيس (بالإنجليزية: Keith ', 'تعلم في Italy High School [الإ']
Found 1 paragraphs for https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AF%D9%88%D8%B1%D9%8A_%D8%A7%D9%84%D9%86%D9%85%D8%B3%D8%A7%D9%88%D9%8A_1921%E2%80%9322 : ['الدوري النمساوي الممتاز 1921–2']
Found 1 paragraphs for https://ar.wikipedia.org/wiki/%D8%B9%D8%AC%D9%8A%D8%A8_(%D8%B4%D9%86%D8%A7%D8%B5) : ['عجيب منطقة سكنية تقع في ولاية ']
Found 13 paragraphs for https://ar.wikipedia.org/wiki/%D9%88%D9%8A%D9%84%D9%8A%D8%A7%D9%85_%D8%A8%D9%86%D8%AF%D9%8A : ['كان وليام بوتنام «بيل» بوندي (', 'بعد تركه الخدمة الحكومية في عا', 'ولد بوندي في عام 1917 ونشأ في ', 'بعد التحاقه بكلية جروتون وجامع', 'بعد تخرجه من كلية الحقوق في عا', 'في أوائل الخمسينيات من القرن ا', 'استمر بوندي في العمل كمستشار ل', 'كما التحق شقيقه، ماكجيري بوندي', 'كان بيل بوندي إلى حد ما على يس', 'شغل بوندي منصب الأمين العام ال', 'كان عمل بوندي الأكثر شهرة هو ك',

ERROR:root:Timeout when trying to find interlanguage links for https://ar.wikipedia.org/wiki/1998_%D9%81%D9%8A_%D9%83%D8%B1%D9%88%D8%A7%D8%AA%D9%8A%D8%A7



Found 2 paragraphs for https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%B2%D8%AD%D9%8A%D8%A9_(%D8%A8%D9%86%D9%8A_%D8%B3%D8%B9%D8%AF) : ['الزحية هي إحدى قرى عزلة بني شد', '']
Found 3 paragraphs for https://ar.wikipedia.org/wiki/%D9%85%D8%B9%D8%AF_%D9%81%D9%8A%D8%A7%D8%B6 : ['معد فياض هو كاتب وصحفي وإعلامي', 'ألف معد فياض العديد من المؤلفا', '']


ERROR:root:Timeout when trying to find interlanguage links for https://ar.wikipedia.org/wiki/%D9%85%D8%B9%D8%AF_%D9%81%D9%8A%D8%A7%D8%B6



Found 1 paragraphs for https://ar.wikipedia.org/wiki/%D8%B3%D8%AA%D9%8A%D9%81%D8%A7%D9%86%D9%8A_%D9%88%D9%8A%D8%B1%D9%8A%D9%86%D8%BA : ['ستيفاني ويرينغ (بالإنجليزية: S']
Found 2 paragraphs for https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D8%A7%D9%81%D8%B8%D8%A9_%D9%8A%D9%88%D9%86%D8%AA%D8%A7%D9%86 : ['محافظة يونتان هي محافظة تقع في', '\u200c']
Found 1 paragraphs for https://ar.wikipedia.org/wiki/%D8%B3%D8%A7%D8%B1%D8%A9_%D8%AC%D9%8A%D9%86_%D9%81%D9%8A%D8%B1%D8%BA%D8%B3%D9%88%D9%86 : ['سارة جين فيرغسون (بالإنجليزية:']
Found 1 paragraphs for https://ar.wikipedia.org/wiki/%D8%B1%D9%88%D9%83%D9%8A_%D8%AC%D9%88%D9%86%D8%B3%D9%88%D9%86 : ['روكي جونسون (بالإنجليزية: Rock']
Found 1 paragraphs for https://ar.wikipedia.org/wiki/%D8%B9%D9%82%D9%8A%D8%A7%D9%86 : ['العِقْيَان[2] أي الذهب، للون أ']
Found 1 paragraphs for https://ar.wikipedia.org/wiki/%D9%8A%D9%88_%D8%A5%D9%81_%D8%B3%D9%8A_144 : ['يو إف سي 144: إدغار ضد هندرسون']
Found 1 paragraphs for https://ar.wikipedia.org/wiki/%D8%B8%D9%84_%D9%85%